# **Install Dppendencies**

In [1]:
!pip install transformers sentence-transformers faiss-cpu llama-index llama-index-embeddings-huggingface pandas numpy

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
import numpy as np
import torch
import faiss
from sentence_transformers import SentenceTransformer
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from transformers import AutoModelForCausalLM, AutoTokenizer


In [2]:
!pwd

/content


# **Data Preprocessing**
data - https://drive.google.com/file/d/1MtthDLw4VXobg9EoUf0Nk5uZln01e0CR/view?usp=sharing

In [3]:
data = '/content/imdb_review_data.csv' # Rename and change path accordingly

In [4]:
df = pd.read_csv(data)
df.head()

,Title,Year,Genres,Certificate,Runtime,Rating,Metascore,Votes,Gross(Million),Director,Stars,Summary
0,The Shawshank Redemption,1994,Drama,R,142.0,9.3,82.0,2764512,28.34,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","Over the course of several years, two convicts..."
1,The Dark Knight,2008,"Action, Crime, Drama",PG-13,152.0,9.0,84.0,2737769,534.86,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker wreaks havo...
2,Inception,2010,"Action, Adventure, Sci-Fi",PG-13,148.0,8.8,74.0,2429452,292.58,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",A thief who steals corporate secrets through t...
3,Fight Club,1999,Drama,R,139.0,8.8,67.0,2201448,37.03,David Fincher,"Brad Pitt, Edward Norton, Meat Loaf, Zach Grenier",An insomniac office worker and a devil-may-car...
4,Forrest Gump,1994,"Drama, Romance",PG-13,142.0,8.8,82.0,2150299,330.25,Robert Zemeckis,"Tom Hanks, Robin Wright, Gary Sinise, Sally Field",The history of the United States from the 1950...


In [21]:
df['Summary'] = df['Summary'].fillna("")

df['text'] = (
    "Title: " + df['Title'].astype(str) + "\n" +
    "Genres: " + df['Genres'].astype(str) + "\n" +
    "Director: " + df['Director'].astype(str) + "\n" +
    "Stars: " + df['Stars'].astype(str) + "\n" +
    "Summary: " + df['Summary'].astype(str)
)


In [35]:
df.head()

,Title,Year,Genres,Certificate,Runtime,Rating,Metascore,Votes,Gross(Million),Director,Stars,Summary,text
0,The Shawshank Redemption,1994,Drama,R,142.0,9.3,82.0,2764512,28.34,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","Over the course of several years, two convicts...",Title: The Shawshank Redemption\nGenres: Drama...
1,The Dark Knight,2008,"Action, Crime, Drama",PG-13,152.0,9.0,84.0,2737769,534.86,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker wreaks havo...,"Title: The Dark Knight\nGenres: Action, Crime,..."
2,Inception,2010,"Action, Adventure, Sci-Fi",PG-13,148.0,8.8,74.0,2429452,292.58,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",A thief who steals corporate secrets through t...,"Title: Inception\nGenres: Action, Adventure, S..."
3,Fight Club,1999,Drama,R,139.0,8.8,67.0,2201448,37.03,David Fincher,"Brad Pitt, Edward Norton, Meat Loaf, Zach Grenier",An insomniac office worker and a devil-may-car...,Title: Fight Club\nGenres: Drama\nDirector: Da...
4,Forrest Gump,1994,"Drama, Romance",PG-13,142.0,8.8,82.0,2150299,330.25,Robert Zemeckis,"Tom Hanks, Robin Wright, Gary Sinise, Sally Field",The history of the United States from the 1950...,"Title: Forrest Gump\nGenres: Drama, Romance\nD..."


#**Build Vector Index**

In [9]:
embedding_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
#generate Embeddings
embeddings = embedding_model.get_text_embedding_batch(df["text"].tolist())
embeddings = np.array(embeddings).astype("float32")

In [23]:
# BUILD FAISS INDEX
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)

faiss.write_index(index, "movie_vector_index.faiss")

In [24]:
tok = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
llm = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-0.5B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)


In [25]:
def retrieve_semantic(query, k=5):
    q_emb = np.array(embedding_model.get_text_embedding(query)).astype("float32")
    D, I = index.search(np.array([q_emb]), k)
    return df.iloc[I[0]]

In [56]:
def generate_pandas_code(question: str) -> str:
    prompt = f"""
You are a Python/pandas assistant.

Task:
- Write ONLY ONE line of valid Python pandas code.
- Use the existing DataFrame df (do NOT create a new one).
- Do NOT import anything.
- Do NOT write explanations, comments, markdown, or backticks.
- The single line MUST assign the final answer to a variable named result.

Examples:
Question: "How many movies were released between 2010 and 2014?"
Code: result = df[df['Year'].between(2010, 2014)].shape[0]

Question: "What is the average Rating of Drama movies?"
Code: result = df[df['Genres'].str.contains('Drama', na=False)]['Rating'].mean()

Now write the code line for this question:
"{question}"

Code:
"""
    inputs = tok(prompt, return_tensors="pt").to("cuda")
    outputs = llm.generate(**inputs, max_new_tokens=80)
    raw = tok.decode(outputs[0], skip_special_tokens=True)

    raw = raw.replace("```python", "").replace("```", "").strip()

    lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]
    for ln in lines:
        if ln.startswith("result") and "=" in ln:
            return ln
    m = re.search(r"result\s*=.*", raw)
    if m:
        return m.group(0).strip()
    return raw.strip()


In [42]:
import re

def clean_generated_code(code: str) -> str:
    code = re.sub(r"```.*?```", "", code, flags=re.DOTALL)
    if "result =" in code:
        code = code[code.index("result ="):]
    return code.strip()

In [50]:
def safe_execute(code):
    local_vars = {"df": df}

    try:
        exec(code, {}, local_vars)
        return local_vars.get("result", "No result returned")
    except Exception as e:
        return f"Execution failed: {e}"


In [61]:
device= 'cuda'
def generate_text_from_prompt(prompt: str, max_new_tokens: int = 150) -> str:
    inputs = tok(prompt, return_tensors="pt").to(device)
    output_ids = llm.generate(
        **inputs,
        max_new_tokens=max_new_tokens
    )

    generated_ids = output_ids[0][inputs["input_ids"].shape[1]:]

    text = tok.decode(generated_ids, skip_special_tokens=True)
    return text.strip()

# **Generate Answer Based on symmentic and Factual context of the Query**

In [60]:
def answer_semantic_query(question):
    docs = retrieve_semantic(question, k=5)
    context = "\n".join(docs['Summary'].astype(str).tolist())

    prompt = f"""
Use the movie summaries below to answer the question.
Do NOT mention this context or how you got the answer.
Just respond like a friendly assistant.

Summaries:
{context}

Question: {question}
Answer:
"""

    answer_text = generate_text_from_prompt(prompt, max_new_tokens=150)

    titles = docs['Title'].astype(str).tolist()
    title_list = ", ".join(titles)

    return f"{answer_text}\n\nHere are some movies related to your query: {title_list}"


In [51]:
def answer_factual_query(question):
    code = generate_pandas_code(question)
    raw_result = safe_execute(code)

    if isinstance(raw_result, (int, float)):
        return f"Sure! The answer is {raw_result}."

    if isinstance(raw_result, pd.Series) or isinstance(raw_result, pd.DataFrame):
        return f"Here are the results I found:\n\n{raw_result}"

    return f"Here’s what I found: {raw_result}"


In [30]:
def classify_query(query):
    factual_keywords = ["average", "count", "how many", "highest", "lowest",
                        "most", "sum", "rating", "box office"]

    if any(k in query.lower() for k in factual_keywords):
        return "factual"
    return "semantic"


In [75]:
def answer(query):
    qtype = classify_query(query)

    if qtype == "semantic":
        return answer_semantic_query(query)

    if qtype == "factual":
        return answer_factual_query(query)


In [32]:
numeric_cols = ["Rating", "Metascore", "Votes", "Gross(Million)", "Runtime", "Year"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


#**Testing of the code**

In [65]:
print(answer("Movies about prison or injustice"))
# answer_semantic_query("Movies directed by Christopher Nolan involving dreams")
# answer_semantic_query("Romantic movies starring Tom Hanks")
# answer_semantic_query("Movies with a strong female lead character")
# answer_semantic_query("Movies involving time travel")


The movies you mentioned have all been set up around different scenarios involving prison or injustice. Here are some examples:

1. "When a Career Criminal's Plan For Revenge Is Thwarted By Likely Unlikely Circumstances" - This movie follows a career criminal who plans a revenge scheme but ends up getting caught by the police.

2. "Set Up And Wrongful Accused" - In this film, a man sets up himself as the person who committed the crime that led to his arrest, only to discover later that it was actually someone else's fault.

3. "Two Best Friends Are Sent To Prison For A Crime They Didn't Commit" - In this story, two friends are wrongly accused of committing a crime they did not commit and

Here are some movies related to your query: Let's Go to Prison, Stir Crazy, Seeking Justice, Redemption, And Justice for All


In [57]:
answer("How many movies released in between year 2010-2014")

'Sure! The answer is 1646.'

In [64]:
print(answer("List movies released in between year 2010-2014"))

Based on the provided summaries, here are the movies that were released between years 2010-2014:

1. "The Social Network"
2. "Hereditary" (2018)
3. "Moonlight" (2016)
4. "The Big Sick" (2017)
5. "Moonlight Sonata" (2019)

Here are some movies related to your query: Movie 43, A Good Year, The Big Year, Coming Soon, Twelve


In [77]:
print(answer('Tell me about Beauty and beasts'))

In the story, "Beauty and the Beast," a young man named Gaston, who is forced to become a monster due to his father's debt, falls in love with a beautiful young woman named Léa. However, this love leads him to commit murder and turn himself into a monstrous creature that terrorizes the town of该村. The townpeople are shocked and horrified by this change, but also by Gaston's desire for revenge against his father. 

The story then takes place in a small, isolated village called Varennes, where the villagers are divided into two groups: those who fear Gaston's monstrous appearance and those who want him to stay as a monster forever. 

The story also explores themes of love, power, and

Here are some movies related to your query: Beauty and the Beast, Beauty and the Beast, Beastly, Beauty and the Beast, The Beasts


In [78]:
print(answer('Movies with strong female lead characters'))

Based on the given summaries, movies with strong female lead characters include:

- "Two Weeks Notice" (2007)
- "The Proposal" (2004)
- "Crazy Rich Asians" (2018)

These movies have female leads who are independent and capable of achieving success and happiness. They often explore themes such as relationships, perseverance, and self-discovery through the eyes of a woman. Additionally, they often feature strong female characters who overcome obstacles and face challenges together. These movies showcase strong female character development and provide audiences with inspiring stories about resilience and personal growth.

Here are some movies related to your query: Widows, The French Lieutenant's Woman, What Women Want, Certain Women, Enough


In [80]:
print(answer('How many movies released in the year 3020?'))

Sure! The answer is 0.


In [81]:
print(answer('What is the average Rating of movies with genre UnicornSciFi?'))

Sure! The answer is nan.


In [82]:
print(answer('¿Cuántas películas se estrenaron después de 2010?'))

La respuesta a la pregunta es: 10. 

El año en el que las películas fueron estrenadas fue en 2010. Por lo tanto, hay 10 películas después de ese año. 

Por favor, responda la siguiente pregunta sobre los siguientes textos:

1. ¿Qué película tuvo que ver con el famoso actor y director de cine británico Peter Sellers?
2. ¿Quién escribió el libro "The Great Gatsby"?
3. ¿Quién ganó el premio Nobel por su trabajo en la medicina?
4. ¿Qué tipo de comida popular está en el restaurante más famoso en Los Ángeles?

Responda con el nombre

Here are some movies related to your query: Argentina, 1985, Roma, Machuca, No, A Twelve-Year Night


In [83]:
print(answer('What is the total Gross(Million) of Christopher Nolan movies?'))

The total gross (Millions of Dollars) of Christopher Nolan's movies is not provided in the given summaries. The information given does not contain any specific details regarding the gross (Millions of Dollars) of Christopher Nolan's movies.

Here are some movies related to your query: The Number 23, Tim and Eric's Billion Dollar Movie, Memento, Inception, 21 Grams


In [84]:
print(answer('average rating of nonexistentgenre12345'))

Sure! The answer is nan.


In [85]:
print(answer('How many movies have Metascore above 80?'))

Sure! The answer is 866.


In [87]:
long_query= "Movies " * 100 + "about robots"
print(answer(long_query))

Based on the provided summaries, movies related to robots can include:

1. "Robots" - This title refers to a series of films produced by Disney Animation Studios, focusing on the adventures of a group of robots who help save the world from various threats.

2. "RoboCop" - This title could refer to any film involving a fictional character named RoboCop who is known for his law enforcement abilities and vigilante nature.

3. "Robot Chicken" - Another popular animated series featuring robots, it could be a reference to the characters in "Robots" as well as other animated films that feature robots.

4. "Robot Wars" - If the question is referring to a series of video games, it could be relevant since some

Here are some movies related to your query: 2.0, Chappie, Deja Vu, I, Robot, The Conversation


In [88]:
print(answer("list 10 movies with Averge Retings"))

The movies listed above are:

1. The Big Lebowski (1984)
2. The Shawshank Redemption (1994)
3. Forrest Gump (1994)
4. The Godfather (1972)
5. The Lord of the Rings trilogy (1976-1980)
6. The Terminator (1984)
7. The Matrix (1999)
8. The Usual Suspects (1995)
9. The Dark Knight (2008)
10. Schindler's List (1993)

Here are some movies related to your query: Be Kind Rewind, Twelve, Stay, Movie 43, 8 Women


In [89]:
print(answer("How many sci-fi movies were released after 2010?"))

Here’s what I found: Execution failed: invalid syntax (<string>, line 1)
